In [2]:
import pandas as pd
import numpy as np
import datetime
import random
from transaction_data_generator import generate_transaction_data
from plotnine import *
from functools import partial
from typing import List, Set, Dict, Tuple, Optional


## Load and explore data

In [3]:
transactions = generate_transaction_data()

In [4]:
transactions.head(5)

,Beneficiary / Originator,Payment Details,Debit,Credit,Booking date,Currency
0,Edeka 1452 BERLIN//BERLIN/DE 02-11-2020T1,Edeka 1452 BERLIN//BERLIN/DE 02-11-2020T1,-55.46,NaN,09/12/2019,EUR
1,Rewe SAGT DANKE,Rewe SAGT DANKE,-0.14,NaN,09/28/2019,EUR
2,Edeka 1452 BERLIN//BERLIN/DE 02-11-2020T1,Edeka 1452 BERLIN//BERLIN/DE 02-11-2020T1,-0.43,NaN,12/09/2019,EUR
3,Lidl 124 DE,Lidl 124 DE,-5.30,NaN,12/22/2019,EUR
4,ALDI SAGT DANKE 128 041//Berlin/DE,ALDI SAGT DANKE 128 041//Berlin/DE,-16.38,NaN,05/16/2019,EUR


In [5]:
transactions.dtypes

Beneficiary / Originator     object
Payment Details              object
Debit                       float64
Credit                      float64
Booking date                 object
Currency                     object
dtype: object

We see that "Booking date" is an object, not a date. We will need to change this.

Does Debit meet Credit? 


In [6]:
transactions[['Debit','Credit']].sum()

Debit    -33186.13
Credit    39000.00
dtype: float64

## Date preparation

So, the very first step is to correctly treat date column. In addition we may want extract day, month, year, calendar week and weekday from it in order to conveniently aggregated data later:

In [7]:
transactions['Booking date'] = pd.to_datetime(transactions['Booking date'])

In [8]:
transactions['day'] = transactions['Booking date'].dt.day
transactions['month'] = transactions['Booking date'].dt.month
transactions['year'] = transactions['Booking date'].dt.year
transactions['weekday'] = transactions['Booking date'].dt.weekday
transactions["week"] = transactions["Booking date"].dt.isocalendar().week



In [9]:
transactions.groupby(['year','month'])[['Debit','Credit']].sum()

Debit  Credit
year month                 
2019 1     -2437.86  3000.0
     2     -2314.10  3000.0
     3     -2861.72  3000.0
     4     -2608.99  3000.0
     5     -3151.01  3000.0
     6     -2260.02  3000.0
     7     -2460.17  3000.0
     8     -2477.27  3000.0
     9     -2527.01  3000.0
     10    -3116.39  3000.0
     11    -2472.01  3000.0
     12    -3199.58  3000.0
2020 1     -1300.00  3000.0

Now we can conveniently check expenses by month:


In [10]:
grouped = transactions.groupby(['year','month']).sum().reset_index()

In [11]:
grouped

,year,month,Debit,Credit,day,weekday,week
0,2019,1,-2437.86,3000.0,548,107,99
1,2019,2,-2314.10,3000.0,355,127,228
2,2019,3,-2861.72,3000.0,692,133,457
3,2019,4,-2608.99,3000.0,642,116,671
4,2019,5,-3151.01,3000.0,400,93,570
5,2019,6,-2260.02,3000.0,324,93,665
6,2019,7,-2460.17,3000.0,522,78,1084
7,2019,8,-2477.27,3000.0,541,102,1189
8,2019,9,-2527.01,3000.0,513,125,1234
9,2019,10,-3116.39,3000.0,569,114,1545


## Types parsing

For the further analysis we need to differentiate different transaction types, i.e. which expense
is related to grocery shopping, which is leisure and so on. In principle, one can train a classifier to solve this problem,
but in such case we will need labeled data. The easiest way forward is a rule-based approach for classification. E.g. we know 
that if Payment Details contain names such "Lidl", "Edeka" or "Rewe" (typical supermarkets in Germany), this is most likely a grocery shopping. If you can find "Booking.com" or "Lufthansa" - this is something to do with travelling. You can always adapt this rule-based approach to your specific data.

In [12]:
types= ['grocery', 'fashion', 'shopping', 'travel', 'rent', 'unknown']
types_mapping = {}
types_mapping['grocery'] = [
    "lidl",
    "rewe",
    "edeka",
    "aldi"
]

types_mapping['fashion'] = [
    "zalando",
    "h&m"
]

types_mapping['shopping'] = [
    "amazon",
    "decathlon",
    "mediamarkt"
]
types_mapping['travel'] = [
    "easyjet",
    "lufthansa",
    "booking.com",
]

types_mapping['rent'] = ["landlord"]

Now we need to go row by row and see, which type we should assigin to the observed transaction. If we have no idea - we assign "unknown" value. 
I decide to do it with help of pandas apply and a simple function below. In addition, we also add a function, to assign an entity to transaction, i.e. shop or beneficiary which we matched based on defined rules. This will allow in future e.g. see expenses by specific retailer.

In [13]:
def assign_type(row, types_mapping: Dict[str, List[str]], info_columns: List[str]):
    matching = []
    for c in info_columns:
        if not pd.isna(row[c]):
            matching.extend(
                [
                    s
                    for s in types_mapping
                    if any(xs in row[c].lower() for xs in types_mapping[s])
                ]
            )

    matching.append("unknown")

    return matching[0]


def assign_entity(row, types_mapping: Dict[str, List[str]], info_columns: List[str]):
    matched_entity = []
    for c in info_columns:
        if not pd.isna(row[c]):

            matched_entity.extend(
                [
                    [xs for xs in types_mapping[s] if xs in row[c].lower()][0]
                    for s in types_mapping
                    if any(xs in row[c].lower() for xs in types_mapping[s])
                ]
            )
    matched_entity.append("unknown_entity")

    return matched_entity[0]

Here, we've noticed in data that information about shop can be in either 'Payment Details' or in "Beneficiary / Originator" columns.
That's why we look in several columns contained in "info_columns" list. If value is not nan in each column, we check if any of the type-specific keywords (e.g. "Lidl" shop name) is contained in the column value. If yes - we add this class in the matching. Potentially, there can be several matchings for some reasons. That's why we append all of them and return the first entry only (simple conflict resolution). And we always add 'unknown' type for the case if no matching was found.

What's left is to prepare partial (because apply expects 1-argument function) and create a new column in the transactions dataframe:

In [14]:
f = partial(assign_type, types_mapping=types_mapping, info_columns=["Beneficiary / Originator", "Payment Details"])
transactions['type'] = transactions.apply(f, axis=1)
f = partial(assign_entity, types_mapping=types_mapping, info_columns=["Beneficiary / Originator", "Payment Details"])
transactions['entity'] = transactions.apply(f, axis=1)

In [15]:
transactions.head(5)

,Beneficiary / Originator,Payment Details,Debit,Credit,Booking date,Currency,day,month,year,weekday,week,type,entity
0,Edeka 1452 BERLIN//BERLIN/DE 02-11-2020T1,Edeka 1452 BERLIN//BERLIN/DE 02-11-2020T1,-55.46,NaN,2019-09-12,EUR,12,9,2019,3,37,grocery,edeka
1,Rewe SAGT DANKE,Rewe SAGT DANKE,-0.14,NaN,2019-09-28,EUR,28,9,2019,5,39,grocery,rewe
2,Edeka 1452 BERLIN//BERLIN/DE 02-11-2020T1,Edeka 1452 BERLIN//BERLIN/DE 02-11-2020T1,-0.43,NaN,2019-12-09,EUR,9,12,2019,0,50,grocery,edeka
3,Lidl 124 DE,Lidl 124 DE,-5.30,NaN,2019-12-22,EUR,22,12,2019,6,51,grocery,lidl
4,ALDI SAGT DANKE 128 041//Berlin/DE,ALDI SAGT DANKE 128 041//Berlin/DE,-16.38,NaN,2019-05-16,EUR,16,5,2019,3,20,grocery,aldi


## Extending types

In terms of dealing with your data, one can always refine the rules. Let's check how many of transactions are rendered as unknown:


In [16]:
transactions.groupby('type')[['Debit']].count()

,Debit
type,
fashion,22
grocery,291
rent,13
shopping,43
travel,4
unknown,40


If we look at those, we can see that indeed I forgot several often transaction classes:


In [17]:
transactions[transactions['type']=='unknown'].head(10)

,Beneficiary / Originator,Payment Details,Debit,Credit,Booking date,Currency,day,month,year,weekday,week,type,entity
5,ROSSMANN 124,ROSSMANN 124,-51.46,NaN,2019-12-21,EUR,21,12,2019,5,51,unknown,unknown_entity
7,ROSSMANN 124,ROSSMANN 124,-38.21,NaN,2019-02-09,EUR,9,2,2019,5,6,unknown,unknown_entity
13,ROSSMANN 124,ROSSMANN 124,-14.37,NaN,2019-07-22,EUR,22,7,2019,0,30,unknown,unknown_entity
21,ROSSMANN 124,ROSSMANN 124,-34.67,NaN,2019-10-23,EUR,23,10,2019,2,43,unknown,unknown_entity
40,ROSSMANN 124,ROSSMANN 124,-9.02,NaN,2019-12-29,EUR,29,12,2019,6,52,unknown,unknown_entity
47,ROSSMANN 124,ROSSMANN 124,-51.52,NaN,2019-09-25,EUR,25,9,2019,2,39,unknown,unknown_entity
48,ROSSMANN 124,ROSSMANN 124,-9.18,NaN,2019-01-20,EUR,20,1,2019,6,3,unknown,unknown_entity
63,ROSSMANN 124,ROSSMANN 124,-48.05,NaN,2019-11-22,EUR,22,11,2019,4,47,unknown,unknown_entity
64,NaN,Salary,NaN,3000.0,2019-06-01,EUR,1,6,2019,5,22,unknown,unknown_entity
65,NaN,Salary,NaN,3000.0,2019-08-01,EUR,1,8,2019,3,31,unknown,unknown_entity


One of the missing shops in grocery category is "ROSSMANN". I also forgot about salary:



In [18]:
types_mapping['grocery'].append('rossmann')
types_mapping['salary'] = ["salary"]

In [19]:
transactions['type'] = transactions.apply(f, axis=1)
transactions.groupby('type').size()

type
aldi            47
amazon          28
booking.com      3
decathlon       11
easyjet          1
edeka          106
landlord        13
lidl            83
mediamarkt       4
rewe            55
rossmann        40
salary          13
zalando         22
dtype: int64

## Code
All the functionallity above is also present in preparation module as functions. You can prepare your own rules and use the same functionality
